# Jonathan Halverson
# Monday, November 20, 2017
# Transfer learning: Part 2

## Part 2: Use the hidden layers from the first model in a new model

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('halverson')

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


We must subtract 5 from the class labels since Tensorflow expects classes to begin at 0.

In [3]:
def filter_digits(A, b, i, j):
     msk = (b <= j) & (b > i)
     return A[msk].copy(), (b[msk] - 5).copy()

In [4]:
images_train_04, labels_train_04 = filter_digits(mnist.train.images, mnist.train.labels, 0, 4)
images_test_04, labels_test_04 = filter_digits(mnist.test.images, mnist.test.labels, 0, 4)

In [5]:
def fetch_batch(A, b, batch_size):
     # could use randint and bootstrapping
     indices = np.random.choice(range(A.shape[0]), size=batch_size, replace=False)
     return A[indices], b[indices]

In [6]:
def one_hundred_each(A, b):
     indices = []
     for i in range(5, 10):
          indices_of_i = np.where(b == i)[0]
          indices.extend(list(np.random.choice(indices_of_i, size=100, replace=False)))
     return A[indices], b[indices] - 5

In [7]:
images_train_59_100, labels_train_59_100 = one_hundred_each(mnist.train.images, mnist.train.labels)
images_test_59, labels_test_59 = filter_digits(mnist.test.images, mnist.test.labels, 5, 9)

In [8]:
np.bincount(labels_train_59_100)

array([100, 100, 100, 100, 100])

Add the previous graph to the default graph:

In [9]:
tf.reset_default_graph()

In [10]:
# load the graph but not the values yet
saver = tf.train.import_meta_graph('/tmp/adam_five_layers.ckpt.meta')

In [11]:
for op in tf.get_default_graph().get_operations():
     if ('eval' in op.name): print (op.name)

eval/InTopK
eval/Cast
eval/Const
eval/Mean


In [12]:
for op in tf.get_default_graph().get_operations():
     print (op.name)

X
y
hidden1/kernel/Initializer/truncated_normal/shape
hidden1/kernel/Initializer/truncated_normal/mean
hidden1/kernel/Initializer/truncated_normal/stddev
hidden1/kernel/Initializer/truncated_normal/TruncatedNormal
hidden1/kernel/Initializer/truncated_normal/mul
hidden1/kernel/Initializer/truncated_normal
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/Const
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden_layers/hidden1/MatMul
hidden_layers/hidden1/BiasAdd
hidden_layers/hidden1/Elu
hidden2/kernel/Initializer/truncated_normal/shape
hidden2/kernel/Initializer/truncated_normal/mean
hidden2/kernel/Initializer/truncated_normal/stddev
hidden2/kernel/Initializer/truncated_normal/TruncatedNormal
hidden2/kernel/Initializer/truncated_normal/mul
hidden2/kernel/Initializer/truncated_normal
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/Const
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
hidden_layers/hidden2/MatMul


In [13]:
n_hidden4 = 100  # new layer
n_outputs = 5  # new layer

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden3 = tf.get_default_graph().get_tensor_by_name("hidden_layers/hidden3/Elu:0")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.elu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, name="new_outputs")

with tf.name_scope("new_loss"):
     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
     loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("new_eval"):
     correct = tf.nn.in_top_k(new_logits, y, 1)
     accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

# Note that the optimizer needed to be given a new name
with tf.name_scope("new_train"):
     learning_rate = 0.001
     optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name="Adam2")
     training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [14]:
def fetch_batch(A, b, batch_size):
     # could use randint and bootstrapping
     indices = np.random.choice(range(A.shape[0]), size=batch_size, replace=False)
     return A[indices], b[indices]

In [19]:
n_epochs = 100
batch_size = 200

In [21]:
with tf.Session() as sess:
     init.run()
     saver.restore(sess, '/tmp/adam_five_layers.ckpt') # load the values
     for epoch in range(n_epochs + 1):
          for iteration in range(images_train_59_100.shape[0] // batch_size):
               X_batch, y_batch = fetch_batch(images_train_59_100, labels_train_59_100, batch_size)
               sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
          accuracy_val = accuracy.eval(feed_dict={X:images_test_59, y:labels_test_59})
          if (epoch % 10 == 0): print(epoch, "Test accuracy:", accuracy_val)

INFO:tensorflow:Restoring parameters from /tmp/adam_five_layers.ckpt
(0, 'Test accuracy:', 0.42680776)
(10, 'Test accuracy:', 0.89216429)
(20, 'Test accuracy:', 0.92441422)
(30, 'Test accuracy:', 0.92718571)
(40, 'Test accuracy:', 0.92945325)
(50, 'Test accuracy:', 0.92794156)
(60, 'Test accuracy:', 0.92819351)
(70, 'Test accuracy:', 0.92794156)
(80, 'Test accuracy:', 0.92945325)
(90, 'Test accuracy:', 0.9297052)
(100, 'Test accuracy:', 0.92894936)


We see that the accuracy isn't very good but we only used 100 samples per digit.